In [1]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import geopandas

import pydeck

In [2]:
folder = fr'V:\switchdrive\Institution\p3'

In [3]:
start_point = (6.1402,46.208742)
delta = 0.02
bbox =  (start_point[0] - delta, start_point[1]-delta, start_point[0] + delta, start_point[1]+delta) 

In [4]:
filepath = os.path.join(folder, 'bases.gpkg')
bases = geopandas.read_file(filepath, bbox=bbox)

filepath = os.path.join(folder, 'facades.gpkg')
facade = geopandas.read_file(filepath, bbox=bbox)

filepath = os.path.join(folder, 'toits.gpkg')
toits = geopandas.read_file(filepath, bbox=bbox)


bases.columns = [s.lower() for s in bases.columns]
facade.columns = [s.lower() for s in facade.columns]
toits.columns = [s.lower() for s in toits.columns]

In [5]:
heat_demand = pd.read_csv('V:\switchdrive\Institution\p3\ge_heat_demand.csv')
norm = mpl.colors.LogNorm(vmin=1, vmax=heat_demand.q_heat_total.max())
cmap = plt.cm.get_cmap('plasma')

In [6]:
MAPBOX_API_KEY = os.environ["MAPBOX_API_KEY"]

In [7]:
toits = toits.merge(heat_demand, on='egid')
cmap_val = cmap(norm(toits.q_heat_total)) 
toits['color'] = (cmap_val* 255).astype(int).tolist()

facade = facade.merge(heat_demand, on='egid')
cmap_val = cmap(norm(facade.q_heat_total)) 
facade['color'] = (cmap_val* 255).astype(int).tolist()

In [8]:
def _f(*args, **kwargs):
    global vals
    vals.append((args, kwargs))


In [9]:
# AWS Open Data Terrain Tiles
TERRAIN_IMAGE = "https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png"

# Define how to parse elevation tiles
ELEVATION_DECODER = {"rScaler": 256, "gScaler": 1, "bScaler": 1 / 256, "offset": -32768}

SURFACE_IMAGE = f"https://api.mapbox.com/v4/mapbox.satellite/{{z}}/{{x}}/{{y}}@2x.png?access_token={MAPBOX_API_KEY}"


# lights = pydeck.LightSettings(
#         lights_position=[start_point[0], start_point[1], 8000, -3.807751, 54.104682, 8000], ambient_ratio=0.4, diffuse_ratio=0.6,
#     lights_strength=0.1
#     )


# lights = pydeck.LightSettings(
#     number_of_lights=1,
#         lights_position=[start_point[0]+1, start_point[1]+1, 80], ambient_ratio=0.4, diffuse_ratio=0.6,
#     lights_strength=0.1
#     )


terrain_layer = pydeck.Layer(
    "TerrainLayer", elevation_decoder=ELEVATION_DECODER, texture=SURFACE_IMAGE, elevation_data=TERRAIN_IMAGE,
    # light_settings=lights

)


bases_layer = pydeck.Layer('GeoJsonLayer', 
                           data=bases, get_fill_color=[255,100,255], pickable=True
                          )
facade_layer = pydeck.Layer('GeoJsonLayer', 
                            data=facade, 
                            get_fill_color='color', 
                            line_width_scale=0.1,
                            pickable=True,
                            # stroked=False,
                            filled=True,
                            wireframe=True,
                            # light_settings=lights

                           )
# facade_layer = pydeck.Layer('GeoJsonLayer', data=facade, get_fill_color='color', pickable=True)
toits_layer = pydeck.Layer('GeoJsonLayer', 
                           data=toits, 
                           get_fill_color='color', 
                           pickable=True,
                           stroked=False,
                           filled=True,
                           wireframe=True,

                          )


INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=start_point[1],
  longitude=start_point[0],
  zoom=14,
  max_zoom=16,
  min_zoom=8,
  pitch=45,
  bearing=0
)


r = pydeck.Deck(
    layers=[
        terrain_layer,
        bases_layer, 
        facade_layer, 
        toits_layer
    ],
    initial_view_state=INITIAL_VIEW_STATE,
    map_style='satellite'
)

r.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"bea…

In [ ]:
def filter_by_bbox(row, west_lng, east_lng, north_lat, south_lat):
    return west_lng < row['lng'] < east_lng and south_lat < row['lat'] < north_lat


def filter_by_viewport(widget_instance, payload):
    try:
        west_lng, north_lat = payload['data']['nw']
        east_lng, south_lat = payload['data']['se']
        filtered_df = df[df.apply(lambda row: filter_by_bbox(row, west_lng, east_lng, north_lat, south_lat), axis=1)]
        text.value = 'Points in viewport: %s' % int(filtered_df.count()['lng'])
    except Exception as e:
        text.value = 'Error: %s' % e

r.deck_widget.on_view_state_change(filter_by_viewport)


In [ ]:
{
    'type': 'deck-view-state-change-event',
    'data': {
        'width': int,
        'height': int,
        'latitude': float,
        'longitude': float,
        'zoom': float,
        'bearing': float,
        'pitch': float,
        'altitude': float,
        'maxZoom': float,
        'minZoom': float
        'maxPitch': float,
        'minPitch': float,
        'nw': [float, float],
        'se': [float, float]
    }
}